## Explore datasets

Tip: Use the try-it-out feature to get a feeling for the API: https://www.data.gouv.fr/fr/dataservices/explore-api-v2-50/

In [6]:
import requests

def get_available_datasets(offset=0, limit=5):
    """Get available datasets from the OFGL API with pagination
    
    Args:
        offset (int): Number of items to skip
        limit (int): Number of items to return
    """
    base_url = "https://data.ofgl.fr/api/explore/v2.1"
    endpoint = "/catalog/datasets"
    
    params = {
        'select': '*',  # Get all fields
        'limit': limit,  # Number of items to return
        'offset': offset  # Number of items to skip
    }
    
    try:
        response = requests.get(base_url + endpoint, params=params)
        if response.status_code == 200:
            data = response.json()
            print(f"Showing datasets {offset+1} to {offset+limit} of {data.get('total_count', 0)}")
            return data
        else:
            print(f"Error: {response.status_code}")
            print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Get the next 5 datasets (skip first 5)
datasets = get_available_datasets(offset=75, limit=5)
print(datasets)

Showing datasets 76 to 80 of 79
{'total_count': 79, 'results': [{'visibility': 'domain', 'dataset_id': 'ofgl-base-sdis', 'dataset_uid': 'da_2fcqmv', 'has_records': True, 'features': ['timeserie', 'analyze'], 'attachments': [], 'alternative_exports': [], 'data_visible': True, 'fields': [{'name': 'exer', 'description': 'Exercice budgétaire', 'annotations': {'facet': True, 'facetsort': '-alphanum', 'sortable': True, 'timeserie_precision': 'year'}, 'label': 'Exercice', 'type': 'date'}, {'name': 'categ', 'description': 'Catégorie de collectivités', 'annotations': {}, 'label': 'Catégorie', 'type': 'text'}, {'name': 'code_region', 'description': 'Numéro INSEE de la région', 'annotations': {'facetsort': '-count'}, 'label': 'Code Région', 'type': 'text'}, {'name': 'nom_region', 'description': 'Nom de la région ou assimilé', 'annotations': {'facet': True, 'facetsort': 'alphanum', 'disjunctive': True}, 'label': 'Nom Région', 'type': 'text'}, {'name': 'code_dep', 'description': 'Numéro INSEE du dé

In [7]:
import requests

def get_internal_datasets():
    # Base URL from api.json
    base_url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets"
    
    # Parameters to search for datasets with "interne" in dataset_id
    params = {
        "limit": 100
    }

    # Make the request
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        for dataset in data['results']:
            print(f"Dataset ID: {dataset['dataset_id']}")
            print(f"Dataset UID: {dataset['dataset_uid']}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

# Execute the function
get_internal_datasets()

Dataset ID: interne-criteres-ei-ofgl-2021
Dataset UID: da_7ums87
Dataset ID: interne-criteres-gfp-ofgl-2020
Dataset UID: da_ac8txt
Dataset ID: table_correspondance_2012_2022_ofgl_large
Dataset UID: da_vm345d
Dataset ID: interne-criteres-ei-ofgl-2023
Dataset UID: da_mowg43
Dataset ID: disponibilite-des-comptes-des-groupements-a-fiscalite-propre
Dataset UID: da_5zmeko
Dataset ID: fiscalite-locale-rei-trace
Dataset UID: da_0bae5n
Dataset ID: correspondance-code-siren-gfp-2024
Dataset UID: da_tbx7td
Dataset ID: interne-criteres-gfp-ofgl-2022
Dataset UID: da_5z7cae
Dataset ID: carto_liste_communes
Dataset UID: da_hcqgli
Dataset ID: detail_compositions_intercommunales_2012_2022
Dataset UID: da_9at4cy
Dataset ID: fpic-ensembles-intercommunaux
Dataset UID: da_ngo1v7
Dataset ID: populations-ofgl-ei
Dataset UID: da_6duh2h
Dataset ID: dotations-regions
Dataset UID: da_1yvh54
Dataset ID: populations-ofgl-gfp
Dataset UID: da_a79zci
Dataset ID: infos-rei
Dataset UID: da_hjgyqy
Dataset ID: ofgl-base-

### Siren (Commune) finances

#### Old version with tabulate

In [6]:
from datetime import datetime
import requests
from tabulate import tabulate


def get_commune_finances_by_siren(
    siren: str,
    year: str = "2023",
    include_details: bool = True
) -> dict:
    """
    Get detailed financial data for a commune using its SIREN number.
    
    Args:
        siren: SIREN number of the commune (9 digits)
        year: Year of data (2016-2023), should be a 4-digit string.
        include_details: Whether to print detailed breakdowns.
        
    Returns:
        dict: Raw API response data
    """
    base_url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets"
    dataset = "ofgl-base-communes-consolidee"
    endpoint = f"{base_url}/{dataset}/records"

    # Ensure year is a properly formatted 4-digit string
    year = str(datetime.strptime(year, "%Y").year)  # Converts to YYYY format

    params = {
        "limit": 100,
        "where": f"siren='{siren}' AND year(exer)='{year}'",
        "order_by": "agregat",
        "select": ("exer,com_name,siren,insee,agregat,montant,montant_bp,montant_ba,"
                  "montant_flux,euros_par_habitant,ptot,rural,montagne,"
                  "touristique,qpv,epci_name")
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()
        results = data.get("results", [])

        if not results:
            print(f"No data found for SIREN {siren} in year {year}")
            return {}

        # Extract basic info from the first record
        basic_info = results[0]
        profile_data = [
            ["Name", basic_info['com_name']],
            ["SIREN", basic_info['siren']],
            ["INSEE Code", basic_info['insee']],
            ["Population", f"{basic_info['ptot']:,}"],
            ["Intercommunality", basic_info['epci_name']]
        ]
        print("\nCommune Profile:")
        print(tabulate(profile_data, tablefmt="grid"))

        if include_details:
            print("\nFinancial Details:")
            financial_data = []
            headers = ["Metric", "Total Amount (€)", "Per Capita (€)", "Primary Budget (€)", 
                      "Annexed Budget (€)", "Flow Amount (€)"]
            for record in results:
                financial_data.append([
                    record['agregat'],
                    f"{record['montant']:,.2f}",
                    f"{record['euros_par_habitant']:,.2f}",
                    f"{record['montant_bp']:,.2f}",
                    f"{record['montant_ba']:,.2f}",
                    f"{record['montant_flux']:,.2f}"
                ])
            print(tabulate(financial_data, headers=headers, tablefmt="grid"))

        return {'raw_data': data}
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return {}

# Example usage:
result = get_commune_finances_by_siren("212102313", "2023")  # Fetch Dijon data for 2023



Commune Profile:
+------------------+-----------------+
| Name             | Dijon           |
+------------------+-----------------+
| SIREN            | 212102313       |
+------------------+-----------------+
| INSEE Code       | 21231           |
+------------------+-----------------+
| Population       | 162,454         |
+------------------+-----------------+
| Intercommunality | Dijon Métropole |
+------------------+-----------------+

Financial Details:
+---------------------------------------------------+--------------------+------------------+----------------------+----------------------+-------------------+
| Metric                                            | Total Amount (€)   | Per Capita (€)   | Primary Budget (€)   |   Annexed Budget (€) |   Flow Amount (€) |
+===================================================+====================+==================+======================+======================+===================+
| Achats et charges externes                        |

#### New version with pandas

In [19]:
from datetime import datetime
import requests
import pandas as pd


def get_commune_finances_by_siren(
    siren: str,
    year: str = "2023",
    include_details: bool = True
) -> dict:
    """
    Get detailed financial data for a commune using its SIREN number.
    
    Args:
        siren: SIREN number of the commune (9 digits)
        year: Year of data (2016-2023), should be a 4-digit string.
        include_details: Whether to print detailed breakdowns.
        
    Returns:
        dict: Raw API response data and processed DataFrames
    """
    base_url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets"
    dataset = "ofgl-base-communes-consolidee"
    endpoint = f"{base_url}/{dataset}/exports/json"

    print(endpoint)
    # Ensure year is a properly formatted 4-digit string
    year = str(datetime.strptime(year, "%Y").year)  # Converts to YYYY format

    params = {
        "where": f"siren='{siren}' AND year(exer)='{year}'",
        "order_by": "agregat",
        "select": ("exer,com_name,siren,insee,agregat,montant,montant_bp,montant_ba,"
                  "montant_flux,euros_par_habitant,ptot,rural,montagne,"
                  "touristique,qpv,epci_name")
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        results = response.json()

        if not results:
            print(f"No data found for SIREN {siren} in year {year}")
            return pd.DataFrame(), {}

        # Create profile DataFrame
        basic_info = results[0]
        profile_df = pd.DataFrame([
            ["Name", basic_info['com_name']],
            ["SIREN", basic_info['siren']],
            ["INSEE Code", basic_info['insee']],
            ["Population", f"{basic_info['ptot']:,}"],
            ["Intercommunality", basic_info['epci_name']]
        ], columns=['Field', 'Value'])
        
        print("\nCommune Profile:")
        print(profile_df.to_string(index=False))

        # Create financial details DataFrame
        financial_df = pd.DataFrame(results)
        financial_df = financial_df[[
            'agregat', 'montant', 'euros_par_habitant', 
            'montant_bp', 'montant_ba', 'montant_flux'
        ]]
        financial_df.columns = [
            'Metric', 'Total Amount (€)', 'Per Capita (€)',
            'Primary Budget (€)', 'Annexed Budget (€)', 'Flow Amount (€)'
        ]
        
        # Format numeric columns
        numeric_columns = financial_df.columns[1:]
        for col in numeric_columns:
            financial_df[col] = financial_df[col].apply(lambda x: f"{x:,.2f}")
        
        if include_details:
            print("\nFinancial Details:")
            print(financial_df.to_string(index=False))

        # Extract key metrics
        total_budget = float(financial_df.loc[financial_df['Metric'] == 'Encours de dette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        total_budget_per_person = float(financial_df.loc[financial_df['Metric'] == 'Encours de dette', 'Per Capita (€)'].iloc[0].replace(',', ''))
        
        gross_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne brute', 'Total Amount (€)'].iloc[0].replace(',', ''))
        operating_revenue = float(financial_df.loc[financial_df['Metric'] == 'Recettes de fonctionnement', 'Total Amount (€)'].iloc[0].replace(',', ''))
        remb_emprunts = float(financial_df.loc[financial_df['Metric'] == "Remboursements d'emprunts hors GAD", 'Total Amount (€)'].iloc[0].replace(',', ''))

        # Get debt service ratio HC
        debt_service = float(financial_df.loc[financial_df['Metric'] == 'Annuité de la dette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        debt_service_to_operating_revenue_ratio = (debt_service / operating_revenue) * 100

        # Get savings metrics (EG, EB, EN)
        management_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne de gestion', 'Per Capita (€)'].iloc[0].replace(',', ''))
        gross_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne brute', 'Per Capita (€)'].iloc[0].replace(',', ''))
        net_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne nette', 'Per Capita (€)'].iloc[0].replace(',', ''))

        # Get savings ratios
        management_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne de gestion', 'Total Amount (€)'].iloc[0].replace(',', ''))
        gross_expenses = float(financial_df.loc[financial_df['Metric'] == 'Recettes totales', 'Total Amount (€)'].iloc[0].replace(',', ''))
        management_savings_ratio = (management_savings / gross_expenses) * 100 # EG/RG
        
        
        gross_savings_ratio = (gross_savings / operating_revenue) * 100 # EB/RF
        net_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne nette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        net_savings_ratio = (net_savings / operating_revenue) * 100 # EN/RF

        metrics = {
            'population': basic_info['ptot'],
            'data_from_year': int(year),
            'total_budget': round(total_budget),
            'total_budget_per_person': round(total_budget_per_person),
            'debt_repayment_capacity': round(total_budget / gross_savings, 1) if gross_savings != 0 else None,
            'debt_ratio': round((total_budget / operating_revenue * 100), 2) if operating_revenue != 0 else None,
            'debt_duration': round((total_budget / remb_emprunts), 1) if remb_emprunts != 0 else None,
            'management_savings_per_capita': round(management_savings_per_capita),
            'management_savings_ratio': round(management_savings_ratio, 2),
            'gross_savings_per_capita': round(gross_savings_per_capita),
            'gross_savings_ratio': round(gross_savings_ratio, 2),
            'net_savings_per_capita': round(net_savings_per_capita),
            'net_savings_ratio': round(net_savings_ratio, 2),
            'debt_service_to_operating_revenue_ratio': round(debt_service_to_operating_revenue_ratio, 2)
        }

        return financial_df, metrics
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return pd.DataFrame(), {}

# Example usage:
df, metrics = get_commune_finances_by_siren("212102313", "2023")  # Fetch Dijon data for 2023
print(metrics)

https://data.ofgl.fr/api/explore/v2.1/catalog/datasets/ofgl-base-communes-consolidee/exports/json

Commune Profile:
           Field           Value
            Name           Dijon
           SIREN       212102313
      INSEE Code           21231
      Population         162,454
Intercommunality Dijon Métropole

Financial Details:
                                           Metric Total Amount (€) Per Capita (€) Primary Budget (€) Annexed Budget (€) Flow Amount (€)
                       Achats et charges externes    48,694,348.21         299.74      48,694,348.21               0.00            0.00
                              Annuité de la dette    15,247,179.91          93.86      15,247,179.91               0.00            0.00
                 Autres dépenses d'investissement     2,345,300.00          14.44       2,345,300.00               0.00            0.00
                Autres dépenses de fonctionnement     4,061,385.94          25.00       4,061,385.94               0.00   

### EPCI finances

In [18]:
from datetime import datetime
import requests
import pandas as pd


def get_epci_finances_by_code(
    epci_code: str,
    year: str = "2023",
    include_details: bool = True
) -> dict:
    """
    Get detailed financial data for an EPCI using its code.
    
    Args:
        epci_code: EPCI code
        year: Year of data (2016-2023), should be a 4-digit string.
        include_details: Whether to print detailed breakdowns.
        
    Returns:
        dict: Raw API response data and processed DataFrames
    """
    base_url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets"
    dataset = "ofgl-base-ei"
    endpoint = f"{base_url}/{dataset}/exports/json"

    print(endpoint)
    # Ensure year is a properly formatted 4-digit string
    year = str(datetime.strptime(year, "%Y").year)  # Converts to YYYY format

    params = {
        "where": f"epci_code='{epci_code}' AND year(exer)='{year}'",
        "order_by": "agregat",
        "select": ("exer,epci_name,epci_code,siren,agregat,montant,montant_gfp,montant_communes,"
                  "montant_flux,euros_par_habitant,ptot,nat_juridique,mode_financement,"
                  "gfp_qpv,reg_name,dep_name")
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        results = response.json()

        if not results:
            print(f"No data found for EPCI {epci_code} in year {year}")
            return pd.DataFrame(), {}

        # Create profile DataFrame
        basic_info = results[0]
        profile_df = pd.DataFrame([
            ["Name", basic_info['epci_name']],
            ["EPCI Code", basic_info['epci_code']],
            ["SIREN", basic_info['siren']],
            ["Legal Status", basic_info['nat_juridique']],
            ["Population", f"{basic_info['ptot']:,}"],
            ["Department", basic_info['dep_name'][0]],
            ["Region", basic_info['reg_name'][0]]
        ], columns=['Field', 'Value'])
        
        print("\nEPCI Profile:")
        print(profile_df.to_string(index=False))

        # Create financial details DataFrame
        financial_df = pd.DataFrame(results)
        financial_df = financial_df[[
            'agregat', 'montant', 'euros_par_habitant', 
            'montant_gfp', 'montant_communes', 'montant_flux'
        ]]
        financial_df.columns = [
            'Metric', 'Total Amount (€)', 'Per Capita (€)',
            'EPCI Amount (€)', 'Communes Amount (€)', 'Flow Amount (€)'
        ]
        
        # Format numeric columns
        numeric_columns = financial_df.columns[1:]
        for col in numeric_columns:
            financial_df[col] = financial_df[col].apply(lambda x: f"{x:,.2f}")
        
        if include_details:
            print("\nFinancial Details:")
            print(financial_df.to_string(index=False))

        # Extract key metrics
        total_budget = float(financial_df.loc[financial_df['Metric'] == 'Encours de dette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        total_budget_per_person = float(financial_df.loc[financial_df['Metric'] == 'Encours de dette', 'Per Capita (€)'].iloc[0].replace(',', ''))
        
        gross_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne brute', 'Total Amount (€)'].iloc[0].replace(',', ''))
        operating_revenue = float(financial_df.loc[financial_df['Metric'] == 'Recettes de fonctionnement', 'Total Amount (€)'].iloc[0].replace(',', ''))
        remb_emprunts = float(financial_df.loc[financial_df['Metric'] == "Remboursements d'emprunts hors GAD", 'Total Amount (€)'].iloc[0].replace(',', ''))

        # Get debt service ratio HC
        debt_service = float(financial_df.loc[financial_df['Metric'] == 'Annuité de la dette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        debt_service_to_operating_revenue_ratio = (debt_service / operating_revenue) * 100

        # Get savings metrics (EG, EB, EN)
        management_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne de gestion', 'Per Capita (€)'].iloc[0].replace(',', ''))
        gross_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne brute', 'Per Capita (€)'].iloc[0].replace(',', ''))
        net_savings_per_capita = float(financial_df.loc[financial_df['Metric'] == 'Epargne nette', 'Per Capita (€)'].iloc[0].replace(',', ''))

        # Get savings ratios
        management_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne de gestion', 'Total Amount (€)'].iloc[0].replace(',', ''))
        gross_expenses = float(financial_df.loc[financial_df['Metric'] == 'Recettes totales', 'Total Amount (€)'].iloc[0].replace(',', ''))
        management_savings_ratio = (management_savings / gross_expenses) * 100 # EG/RG
        
        
        gross_savings_ratio = (gross_savings / operating_revenue) * 100 # EB/RF
        net_savings = float(financial_df.loc[financial_df['Metric'] == 'Epargne nette', 'Total Amount (€)'].iloc[0].replace(',', ''))
        net_savings_ratio = (net_savings / operating_revenue) * 100 # EN/RF
        

        metrics = {
            'population': basic_info['ptot'],
            'data_from_year': int(year),
            'total_budget': round(total_budget),
            'total_budget_per_person': round(total_budget_per_person),
            'debt_repayment_capacity': round(total_budget / gross_savings, 1) if gross_savings != 0 else None,
            'debt_ratio': round((total_budget / operating_revenue * 100), 2) if operating_revenue != 0 else None,
            'debt_duration': round((total_budget / remb_emprunts), 1) if remb_emprunts != 0 else None,
            'management_savings_per_capita': round(management_savings_per_capita),
            'management_savings_ratio': round(management_savings_ratio, 2),
            'gross_savings_per_capita': round(gross_savings_per_capita),
            'gross_savings_ratio': round(gross_savings_ratio, 2),
            'net_savings_per_capita': round(net_savings_per_capita),
            'net_savings_ratio': round(net_savings_ratio, 2),
            'debt_service_to_operating_revenue_ratio': round(debt_service_to_operating_revenue_ratio, 2)
        }

        return financial_df, metrics
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return pd.DataFrame(), {}

# Example usage:
df, metrics = get_epci_finances_by_code("242100410", "2023")  # Dijon Métropole 2023
print(metrics)


https://data.ofgl.fr/api/explore/v2.1/catalog/datasets/ofgl-base-ei/exports/json

EPCI Profile:
       Field                   Value
        Name         Dijon Métropole
   EPCI Code               242100410
       SIREN               242100410
Legal Status                       M
  Population                 261,901
  Department               Côte-d'Or
      Region Bourgogne-Franche-Comté

Financial Details:
                                           Metric Total Amount (€) Per Capita (€) EPCI Amount (€) Communes Amount (€) Flow Amount (€)
                       Achats et charges externes   216,517,782.89         826.72  141,658,911.87       76,433,232.45    1,574,361.43
                              Annuité de la dette    59,412,651.79         226.85   38,427,684.14       20,989,131.89        4,164.24
                 Autres dépenses d'investissement     3,366,885.99          12.86    1,021,110.69        2,345,775.30            0.00
                Autres dépenses de fonctionnement   